#### **Construcción de un Modelo de Scoring de Comportamiento**

**Objetivo:** Crear un modelo que otorgue a cada cliente una puntuación para ordenar a los clientes de acuerdo con su calidad crediticia.


**Revisión y Análisis de los Datos**

**Librerías**

In [1]:
import pandas as pd
import pandas as pd
from pyxlsb import open_workbook
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

**Funciones**

In [2]:
def grafico_dispersion_2_var(df, var1, var2):
    sns.scatterplot(data=df, x=var1, y=var2, hue='default')
    plt.show()

Explicación: Esta función genera un gráfico de dispersión (scatter plot) para dos variables, var1 y var2. Utiliza seaborn para trazar el gráfico y la opción hue='default' colorea los puntos según la variable default, lo que te permite visualizar cómo se distribuyen las observaciones en función de la variable de interés. Este tipo de gráficos son útiles para identificar relaciones entre dos variables y cómo se segmentan por la clase objetivo.

### **1. Carga y Exploración de los Datos**

In [3]:
# Define la ruta al archivo .xlsb
file_path = './BBDD_Scoring.xlsb' 

# Inicializa una lista para almacenar los datos
data = []

# Lee el archivo y extrae las filas
with open_workbook(file_path) as wb:
    with wb.get_sheet(1) as sheet:  # Cambia "1" al índice correcto de la hoja si es necesario
        for row in sheet.rows():
            # Extrae los valores de cada fila
            data.append([item.v for item in row])

# Convierte los datos en un DataFrame
df = pd.DataFrame(data[1:], columns=data[0])  # Usa la primera fila como nombres de columnas

In [4]:
def grafico_dispersion_2_var(df, var1, var2):
    sns.scatterplot(data=df, x=var1, y=var2, hue='default')
    plt.show()

In [5]:
# Muestra las primeras filas para verificar la carga
df.head()

,Cliente,default,Edad,Sal_BurCto_Year,Dias_impago_hip,Dias_impago_hip_year,Dias_mora_hip_year,Ant_hipotecaria,Importe_impago_hip,Importe_impago_hip_year,...,Importe_mora_year,Dias_impago_mes,Dias_impago_tri,Dias_impago_year,Dias_impago_2year,Dias_mora,Dias_mora_tri,Dias_mora_year,Dias_mora_2year,Cuotas_ptmos
0,1.0,0.0,70.42,0.0,0.0,0.0,0.0,102.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1,2.0,0.0,44.28,0.0,0.0,0.0,0.0,108.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,425.81
2,3.0,0.0,46.70,0.0,0.0,0.0,0.0,162.0,0.0,0.0,...,0.0,0.0,3.0,5.0,6.0,0.0,0.0,0.0,0.0,377.76
3,4.0,0.0,68.19,0.0,0.0,0.0,0.0,159.0,0.0,0.0,...,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,853.85
4,5.0,0.0,55.43,0.0,0.0,0.0,0.0,188.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


Se ha decidido **quitar las columnas que solo tienen un valor único**, ya que no aportan información al modelo.

In [6]:
# Calcular número de valores distintos en cada columna
columnas_sin_valores_distintos = df.nunique()[df.nunique() < 2].index
df = df.drop(columnas_sin_valores_distintos, axis=1)
df

,Cliente,default,Edad,Sal_BurCto_Year,Dias_impago_hip_year,Dias_mora_hip_year,Ant_hipotecaria,Importe_impago_hip_year,Dias_imp_ptmopres_Tri,Importe_impago_ptmopers_year,...,Importe_impagado_year,Importe_mora_tri,Importe_mora_year,Dias_impago_tri,Dias_impago_year,Dias_impago_2year,Dias_mora_tri,Dias_mora_year,Dias_mora_2year,Cuotas_ptmos
0,1.0,0.0,70.42,0.0,0.0,0.0,102.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1,2.0,0.0,44.28,0.0,0.0,0.0,108.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,425.81
2,3.0,0.0,46.70,0.0,0.0,0.0,162.0,0.00,0.0,0.0,...,154.99,0.0,0.00,3.0,5.0,6.0,0.0,0.0,0.0,377.76
3,4.0,0.0,68.19,0.0,0.0,0.0,159.0,0.00,0.0,0.0,...,50.56,0.0,0.00,0.0,4.0,4.0,0.0,0.0,0.0,853.85
4,5.0,0.0,55.43,0.0,0.0,0.0,188.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249428,249429.0,1.0,37.69,0.0,35.0,0.0,35.0,2819.09,0.0,0.0,...,2819.09,0.0,21.03,0.0,91.0,91.0,0.0,20.0,20.0,0.00
249429,249430.0,1.0,52.48,0.0,0.0,0.0,15.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,232.28
249430,249431.0,1.0,51.50,0.0,0.0,0.0,15.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,689.01
249431,249432.0,1.0,36.41,0.0,0.0,0.0,13.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,92.33


Explicación:

Se calculan cuántos valores únicos tiene cada columna con df.nunique(). Luego, se filtran las columnas que tienen menos de 2 valores únicos (es decir, aquellas que contienen solo un valor).
Estas columnas se eliminan del DataFrame ya que no aportan valor predictivo al modelo.

In [7]:
# 1. Exploración Inicial

# Convertir a variables categóricas para su posterior análisis
df['Cliente'] = df['Cliente'].astype('object')
df['default'] = df['default'].astype('object')
df['Indicador_tarjeta'] = df['Indicador_tarjeta'].astype('object')
df['Indicador_recibos'] = df['Indicador_recibos'].astype('object')

print("Tipos de datos:\n", df.dtypes)

Tipos de datos:
 Cliente                           object
default                           object
Edad                             float64
Sal_BurCto_Year                  float64
Dias_impago_hip_year             float64
Dias_mora_hip_year               float64
Ant_hipotecaria                  float64
Importe_impago_hip_year          float64
Dias_imp_ptmopres_Tri            float64
Importe_impago_ptmopers_year     float64
Dias_impago_tarjeta_tri          float64
Dias_impago_tarjeta_year         float64
Dias_impago_tarjeta_2year        float64
Cuota_tarjetas                   float64
Indicador_tarjeta                 object
Dias_impago_descubierto_tri      float64
Dias_impago_descubierto_year     float64
Dias_impago_descubierto_2year    float64
Dias_mora_descubierto_2year      float64
Descubierto_year                 float64
Saldo_cta_coriente               float64
Saldo_total                      float64
Saldo_cta_corriente_tri          float64
Saldo_cta_corriente_year         float64

In [8]:
# 2. Resumen Estadístico
print("\nResumen estadístico:\n", df.describe())


Resumen estadístico:
                 Edad  Sal_BurCto_Year  Dias_impago_hip_year  \
count  249433.000000     2.494330e+05         249433.000000   
mean       47.055684     1.433335e+02              1.004498   
std        10.938642     8.428254e+03              8.268045   
min         6.110000     0.000000e+00              0.000000   
25%        39.190000     0.000000e+00              0.000000   
50%        44.950000     0.000000e+00              0.000000   
75%        53.630000     0.000000e+00              0.000000   
max       106.170000     3.707142e+06           1981.000000   

       Dias_mora_hip_year  Ant_hipotecaria  Importe_impago_hip_year  \
count       249433.000000    249433.000000            249433.000000   
mean             0.971058       120.022367                58.625361   
std             41.418899        61.009153              1878.872532   
min              0.000000         0.000000                 0.000000   
25%              0.000000        76.000000            

In [9]:
# 3. Conteo de valores nulos --> NO HAY
print(df.isnull().sum())  

Cliente                          0
default                          0
Edad                             0
Sal_BurCto_Year                  0
Dias_impago_hip_year             0
Dias_mora_hip_year               0
Ant_hipotecaria                  0
Importe_impago_hip_year          0
Dias_imp_ptmopres_Tri            0
Importe_impago_ptmopers_year     0
Dias_impago_tarjeta_tri          0
Dias_impago_tarjeta_year         0
Dias_impago_tarjeta_2year        0
Cuota_tarjetas                   0
Indicador_tarjeta                0
Dias_impago_descubierto_tri      0
Dias_impago_descubierto_year     0
Dias_impago_descubierto_2year    0
Dias_mora_descubierto_2year      0
Descubierto_year                 0
Saldo_cta_coriente               0
Saldo_total                      0
Saldo_cta_corriente_tri          0
Saldo_cta_corriente_year         0
Saldo_depositos_tri              0
Saldo_total_tri                  0
Saldo_total_year                 0
Indicador_Saldo                  0
Ingresos_totales    

In [10]:
# 4. Duplicados

# Detectar filas duplicadas
duplicados = df.duplicated()

# Mostrar el número de filas duplicadas
print(f'Número de filas duplicadas: {duplicados.sum()}')

Número de filas duplicadas: 0


In [11]:
# 5 . Distribución de variables categóricas (columna'default')
print("\nDistribución de valores categóricos:")
print(df['default'].value_counts())


Distribución de valores categóricos:
0.0    249045
1.0       388
Name: default, dtype: int64


**Creación de variables**

In [12]:
df['saldo_<_30k'] = (df['Saldo_total'] < 30000).astype(int)

# tabla con porcentaje de cantidad de personas con saldo menor a 30k y default
tabla = df.groupby('default')['saldo_<_30k'].value_counts(normalize=True).unstack()
display(tabla) # significa que el 97% de las personas con saldo menor a 30k han entrado en default y solo el 

tabla2 = df.groupby('saldo_<_30k')['default'].value_counts(normalize=True).unstack()
display(tabla2)

saldo_<_30k,0,1
default,,
0.0,0.189171,0.810829
1.0,0.033505,0.966495


default,0.0,1.0
saldo_<_30k,,
0,0.999724,0.000276
1,0.998146,0.001854


Explicación:

Se crea una nueva variable binaria saldo_<_30k, que toma el valor 1 si el saldo total es menor que 30,000 y 0 en caso contrario. Esto es útil para segmentar a los clientes según su saldo.

Explicación:

Estas tablas muestran las frecuencias relativas (porcentaje) de saldo_<_30k dentro de cada clase de default y viceversa.
La primera tabla muestra cómo se distribuyen los saldos bajos dentro de los grupos de default, mientras que la segunda tabla muestra cómo el default se distribuye dentro de los grupos de saldo bajo.
Estas tablas son útiles para entender la relación entre el saldo y el comportamiento de pago.

In [13]:
df.loc[df['Ingresos_totales'] == 0, 'Ingresos_totales'] = 1
df['Gastos/Ingresos'] = df['Gastos'] / df['Ingresos_totales']
df['Cuotas_ptmos/Ingresos'] = df['Cuotas_ptmos'] / df['Ingresos_totales']
df['Recibos_denegados_dummy'] = (df['Recibos_denegados'] > 0).astype(int)
df['Ant_hipotecaria_>_100'] = (df['Ant_hipotecaria'] > 100).astype(int)

In [14]:
df['Recibos_denegados_dummy'] = df['Recibos_denegados_dummy'].astype('object')
df['Ant_hipotecaria_>_100'] = df['Ant_hipotecaria_>_100'].astype('object')

Explicación:

Gastos/Ingresos: Ratio entre los gastos e ingresos totales, lo cual puede indicar la capacidad de pago de un individuo.

Cuotas_ptmos/Ingresos: Ratio entre las cuotas de pago mensual y los ingresos totales, indicando la proporción de ingresos destinados a pagos.

Recibos_denegados_dummy: Una variable binaria que indica si una persona tiene recibos denegados (útil para identificar posibles problemas financieros).

Ant_hipotecaria_>_100: Una variable binaria que indica si la antigüedad hipotecaria es mayor a 100 meses.

**Variables de interés (según criterio)**

In [15]:
variables_interes = [
    'default', 'Edad', 'Dias_impago_hip_year', 'Dias_mora_hip_year',
    'Dias_impago_descubierto_year', 'Saldo_total',
    'Saldo_cta_corriente_year', 'Ingresos_totales', 'Gastos', 'Mora_ult5A'
]